Source: ICLR

Year: 2016

Authors: Tom Schaul, John Quan, Ioannis Antonoglou, David Silver

Institutions: Google DeepMind

Prioritized Experience Replay replays transitions with high expected learning progress, as measured by the magnitude of their temporal-difference (TD) error. The prioritization can lead to a loss of diversity, which is alleviated with stochastic prioritization, and introduce bias, which is corrected with importance sampling.

To scale to large memory sizes $N$, it uses a binary heap data structure for the priority queue, for which finding the maximum priority transition when sampling is $O(1)$ and updating priorities (with the new TD-error after a learning step) is $O(\log N)$.

However, greedy TD-error prioritization has several issues.
1. To avoid expensive sweeps over the entire replay memory, TD errors are only updated for the transitions that are replayed. One consequence is that transitions that have a low TD error on first visit may not be replayed for a long time.
2. It is sensitive to noise spikes (e.g. when rewards are stochastic), which can be exacerbated by bootstrapping, where approximation errors appear as another source of noise. 
3. Greedy prioritization focuses on a small subset of the experience: errors shrink slowly, especially when using function approximation, meaning that the initially high error transitions get replayed frequently. This lack of diversity that makes the system prone to over-fitting.

To overcome these issues, a stochastic sampling method is introduced that interpolates between pure greedy prioritization and uniform random sampling. The probability of being sampled is monotonic in a transition’s priority, while guaranteeing a non-zero probability even for the lowest-priority transition. Concretely, the probability of sampling transition $i$ as:
$$P(i)=\frac{P^\alpha_i}{\sum_k p^\alpha_k}$$
where $p_i>0$ is the priority of transition $i$. The exponent $\alpha$ determines how much prioritization is used, with $\alpha=0$ corresponding to the uniform case.

Two variants of prioritization:
1. $p_i=|\delta_i|+\epsilon$
2. $p_i=\frac{1}{rank(i)}$, where $rank(i)$ is the rank of transition $i$ when the replay memory is sorted according to $|\delta_i|$.

Both distributions are monotonic in $|\delta|$, but the latter is likely to be more robust, as it is insensitive to outliers.

To efficiently sample from distribution, the complexity cannot depend on $N$. For the rank-based variant, we can approximate the cumulative density function with a piecewise linear function with $k$ segments of equal probability. The segment boundaries can be precomputed (they change only when $N$ or $\alpha$ change). At runtime, a segment is sampled, and then the transitions are sampled uniformly within it. This works particularly well in conjunction with a minibatch-based learning algorithm: choose $k$ to be the size of the minibatch, and sample exactly one transition from each segment – this is a form of stratified sampling that has the added advantage of balancing out the minibatch (there will always be exactly one transition with high magnitude $\delta$, one with medium magnitude, etc). The proportional variant is different, also admits an efficient implementation based on a ‘sum-tree’ data structure (where every node is the sum of its children, with the priorities as the leaf nodes), which can be efficiently updated and sampled from.

The estimation of the expected value with stochastic updates relies on those updates corresponding to the same distribution as its expectation. Prioritized replay introduces bias because it changes this distribution in an uncontrolled fashion, and therefore changes the solution that the estimates will converge to (even if the policy and state distribution are fixed). The bias can be corrected by importance-sampling (IS) weights
$$w_i=(NP(i))^-\beta$$
that fully compensates for the non-uniform probabilities $P(i)$ if $\beta=1$. These weights can be folded into the Q-learning update by using $w_i\delta_i$ instead of $\delta_i$. For stability reasons, weights are normalized by $\frac{1}{max_i w_i}$ so that they only scale the update downwards.

In typical reinforcement learning scenarios, the unbiased nature of the updates is most important near convergence at the end of training, as the process is highly non-stationary anyway, due to changing policies, state distributions and bootstrap targets; a small bias can be ignored in this context.

It therefore exploits the flexibility of annealing the amount of importance-sampling correction over time, by defining a schedule on the exponent $\beta$ that reaches 1 only at the end of learning. In practice, we linearly anneal $\beta$ from its initial value $\beta_0$ to $1$. Note that the choice of this hyperparameter interacts with choice of prioritization exponent $\alpha$; increasing both simultaneously prioritizes sampling more aggressively at the same time as correcting for it more strongly.